**Library Imports**

In [ ]:
!pip install imblearn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import math
import nltk
from nltk.corpus import words
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.svm import LinearSVC

Mount Google Drive and Load Dataset

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Load data
df = pd.read_csv('/content/drive/MyDrive/password_strength_dataset/cleaned_password_strength.csv')

Mounted at /content/drive


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18192 entries, 0 to 18191
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   strength               18192 non-null  float64
 1   length                 18192 non-null  int64  
 2   digit_count            18192 non-null  int64  
 3   lowercase_count        18192 non-null  int64  
 4   uppercase_count        18192 non-null  int64  
 5   special_count          18192 non-null  int64  
 6   entropy                18192 non-null  float64
 7   sequential_chars       18192 non-null  int64  
 8   dictionary_word_count  18192 non-null  int64  
dtypes: float64(2), int64(7)
memory usage: 1.2 MB


In [ ]:
df.describe()

,strength,length,digit_count,lowercase_count,uppercase_count,special_count,entropy,sequential_chars,dictionary_word_count
count,18192.000000,18192.00000,18192.000000,18192.000000,18192.000000,18192.000000,18192.000000,18192.000000,18192.000000
mean,1.610378,14.87555,4.877144,6.943382,2.112082,0.942502,3.205205,4.640996,0.167051
std,0.551277,5.18104,3.293584,5.153441,3.120194,1.477701,0.639367,1.745288,0.404018
min,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000
25%,1.000000,12.00000,2.000000,3.000000,0.000000,0.000000,2.855389,4.000000,0.000000
50%,2.000000,15.00000,4.000000,6.000000,1.000000,0.000000,3.334679,5.000000,0.000000
75%,2.000000,17.00000,7.000000,10.000000,3.000000,1.000000,3.640224,5.000000,0.000000
max,2.000000,220.00000,39.000000,164.000000,80.000000,21.000000,5.684569,73.000000,8.000000


In [ ]:
df.head()

,strength,length,digit_count,lowercase_count,uppercase_count,special_count,entropy,sequential_chars,dictionary_word_count
0,1.0,8,4,4,0,0,2.500000,2,0
1,1.0,8,4,4,0,0,2.500000,2,1
2,1.0,9,2,7,0,0,2.947703,3,0
3,1.0,8,3,5,0,0,3.000000,2,0
4,1.0,11,1,10,0,0,3.459432,3,0


Split Data for Model Training

In [ ]:
X = df.drop(columns=['strength'])  # Features
y = df['strength']                 # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)


Feature Scaling

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert scaled arrays back to DataFrames (optional but useful)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns)


# over_sampling

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


# Training and Evaluation

# RF

In [ ]:

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(random_state=42)

In [ ]:
# Scale both train and test
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrames with column names
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Train on scaled data
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train_scaled_df, y_train)

# Predict
y_pred = classifier.predict(X_test_scaled_df)       # Show confusion matrix

In [ ]:
print(accuracy_score(y_test, y_pred))              # Accuracy score


0.9963356540857456


In [ ]:
print(classification_report(y_test, y_pred, digits=4))  # Precision, recall, f1-score

              precision    recall  f1-score   support

         0.0     0.9319    0.9622    0.9468       185
         1.0     0.9960    0.9926    0.9943      1766
         2.0     1.0000    1.0000    1.0000      3507

    accuracy                         0.9963      5458
   macro avg     0.9760    0.9849    0.9804      5458
weighted avg     0.9964    0.9963    0.9964      5458



In [ ]:

error_rate = 1 - accuracy_score(y_test, y_pred)
print(f"Error Rate: {error_rate:.4f}")


Error Rate: 0.0037


In [ ]:
# Evaluation
print(confusion_matrix(y_test, y_pred))

[[ 178    7    0]
 [  13 1753    0]
 [   0    0 3507]]


# SVM

In [ ]:
# Scale both train and test
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrames with column names
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Train on scaled data
svc = make_pipeline(StandardScaler(), LinearSVC(random_state=42))
svc.fit(X_train_scaled_df, y_train)

# Predict
y_pred = svc.predict(X_test_scaled_df)       # Show confusion matrix

In [ ]:
print(accuracy_score(y_test, y_pred))              # Accuracy score


0.9836936606815684


In [ ]:
print(classification_report(y_test, y_pred, digits=4))  # Precision, recall, f1-score

              precision    recall  f1-score   support

         0.0     0.9802    0.5351    0.6923       185
         1.0     0.9535    0.9983    0.9754      1766
         2.0     0.9997    1.0000    0.9999      3507

    accuracy                         0.9837      5458
   macro avg     0.9778    0.8445    0.8892      5458
weighted avg     0.9841    0.9837    0.9815      5458



In [ ]:

error_rate = 1 - accuracy_score(y_test, y_pred)
print(f"Error Rate: {error_rate:.4f}")


Error Rate: 0.0163


In [ ]:
# Evaluation
print(confusion_matrix(y_test, y_pred))

[[  99   86    0]
 [   2 1763    1]
 [   0    0 3507]]


# Ensemble Voting

In [ ]:
# Define individual models (ensure SVC is in a pipeline to handle scaling)
svc = make_pipeline(StandardScaler(), LinearSVC(random_state=42))
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Create Voting Ensemble (Hard Voting)
ensemble = VotingClassifier(estimators=[
    ('svc', svc),
    ('rf', rf)
], voting='hard')
ensemble.fit(X_train_scaled_df, y_train)
# Predict
y_pred = ensemble.predict(X_test_scaled_df)

In [ ]:
print(accuracy_score(y_test, y_pred))              # Accuracy score


0.9965188713814584


In [ ]:
print(classification_report(y_test, y_pred, digits=4))  # Precision, recall, f1-score

              precision    recall  f1-score   support

         0.0     0.9323    0.9676    0.9496       185
         1.0     0.9966    0.9926    0.9946      1766
         2.0     1.0000    1.0000    1.0000      3507

    accuracy                         0.9965      5458
   macro avg     0.9763    0.9867    0.9814      5458
weighted avg     0.9966    0.9965    0.9965      5458



In [ ]:

error_rate = 1 - accuracy_score(y_test, y_pred)
print(f"Error Rate: {error_rate:.4f}")


Error Rate: 0.0035


In [ ]:
# Evaluation
print(confusion_matrix(y_test, y_pred))

[[ 179    6    0]
 [  13 1753    0]
 [   0    0 3507]]


# Bagging

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier

# Corrected: use 'estimator' instead of 'base_estimator'
boost_bagging = BalancedBaggingClassifier(estimator=DecisionTreeClassifier(), random_state=42)
boost_bagging.fit(X_train_scaled_df, y_train)


BalancedBaggingClassifier(estimator=DecisionTreeClassifier(), random_state=42)

In [ ]:
y_pred = boost_bagging.predict(X_test_scaled_df)        # Predict on test set

# Evaluation
print(confusion_matrix(y_test, y_pred))            # Show confusion matrix

[[ 183    2    0]
 [  22 1744    0]
 [   0    0 3507]]


In [ ]:
print(accuracy_score(y_test, y_pred))              # Accuracy score


0.9956027849028949


In [ ]:
print(classification_report(y_test, y_pred, digits=4))  # Precision, recall, f1-score

              precision    recall  f1-score   support

         0.0     0.8927    0.9892    0.9385       185
         1.0     0.9989    0.9875    0.9932      1766
         2.0     1.0000    1.0000    1.0000      3507

    accuracy                         0.9956      5458
   macro avg     0.9638    0.9922    0.9772      5458
weighted avg     0.9960    0.9956    0.9957      5458



In [ ]:

error_rate = 1 - accuracy_score(y_test, y_pred)
print(f"Error Rate: {error_rate:.4f}")


Error Rate: 0.0044


# Boosting

In [ ]:
# Training the Random Forest Classifier
boost = EasyEnsembleClassifier()
boost.fit(X_train_scaled_df, y_train)

EasyEnsembleClassifier()

In [ ]:
y_pred = boost.predict(X_test_scaled_df)        # Predict on test set

# Evaluation
print(confusion_matrix(y_test, y_pred))            # Show confusion matrix

[[ 182    3    0]
 [  26 1740    0]
 [   0    0 3507]]


In [ ]:
print(accuracy_score(y_test, y_pred))              # Accuracy score


0.9946866984243312


In [ ]:
print(classification_report(y_test, y_pred, digits=4))  # Precision, recall, f1-score

              precision    recall  f1-score   support

         0.0     0.8750    0.9838    0.9262       185
         1.0     0.9983    0.9853    0.9917      1766
         2.0     1.0000    1.0000    1.0000      3507

    accuracy                         0.9947      5458
   macro avg     0.9578    0.9897    0.9726      5458
weighted avg     0.9952    0.9947    0.9948      5458



In [ ]:

error_rate = 1 - accuracy_score(y_test, y_pred)
print(f"Error Rate: {error_rate:.4f}")


Error Rate: 0.0053
